In [48]:
import requests
import json
import pandas as pd
import numpy as np

In [49]:
# get list posisi
posisi = requests.get('http://127.0.0.1:4000/posisi').json()
posisi_array = [item['nama'] for item in posisi['data']]

In [50]:
# get list kriteria
kriteria = requests.get('http://127.0.0.1:4000/kriteria').json()

In [51]:
# add kecocokan_jurusan criteria
kecocokan_jurusan = {
    'id': 99,
    'nama': 'kecocokan_jurusan',
    'tipe': 'benefit',
    'kepentingan': 'sangat penting',
    'created_at': None,
    'updated_at': None
}
kriteria['data'].append(kecocokan_jurusan)
crit = kriteria['data']

In [52]:
#df kriteria
crit_df = pd.DataFrame(crit)
crit_df = crit_df.drop(['id', 'tipe', 'created_at', 'updated_at'], axis=1)

In [53]:
# get data alternatif
response=requests.get('http://127.0.0.1:4000/hasil/alternatif').json()
df=pd.DataFrame(response['data'])

# split
df = df.applymap(lambda x: x.split(',') if isinstance(x, str) else x)
column_headers = list(df.columns.values)
for i in range(len(column_headers)) :
    df = df.explode(column_headers[i]).reset_index(drop=True)


In [54]:
# get data nilai jurusan
response_2=requests.get('http://127.0.0.1:4000/hasil/jurusan').json()
df_2=pd.DataFrame(response_2['data'])
df_2 = df_2.drop(['id'], axis=1)

melted_df = pd.melt(df_2, id_vars='nama_jurusan', var_name='posisi', value_name='kecocokan_jurusan')

In [55]:
merged_df = pd.merge(df, melted_df, on=['nama_jurusan', 'posisi'], how='left')

In [56]:
spk_df = merged_df.drop(['nama', 'nama_jurusan', 'posisi'], axis=1)

In [57]:
# normalisasi kriteria
for column in spk_df.columns: 
    for index, kriteria in enumerate(crit):
        # display(crit[index]['tipe'])
        if (column == crit[index]['nama']):
            if (crit[index]['tipe'] == 'benefit'):
                spk_df[column] = spk_df[column]  / spk_df[column].abs().max()
            else:
                spk_df[column] = spk_df[column]  / spk_df[column].abs().min()

In [58]:
#replace tingkat kepentingan
dict = {"sangat kurang penting" : '1', "kurang penting" : '2', "menengah": '3', "penting" : '4', "sangat penting": '5'}
crit_df = crit_df.replace({'kepentingan': dict})

In [59]:
#perbaikan bobot
crit_df['kepentingan'] = crit_df['kepentingan'].astype(float)
crit_df['kepentingan'] = crit_df['kepentingan'] / 10

In [60]:
#transpose df
crit_df = crit_df.set_index('nama').T

In [61]:
# pemeringkatan
common_columns = []
for column in spk_df.columns: 
    common_columns.append(column)

spk_df = spk_df[common_columns]
crit_df = crit_df[common_columns]
df_result = spk_df * crit_df.iloc[0]

result_value = df_result.sum(axis=1)
result_value

0     1.908491
1     1.808491
2     2.008491
3     2.205996
4     2.005996
5     1.805996
6     2.344552
7     2.244552
8     2.244552
9     2.311841
10    2.211841
11    1.911841
12    1.740517
13    1.840517
14    2.040517
dtype: float64

In [62]:
result_spk = merged_df[['nama', 'nama_jurusan', 'posisi']].copy()
result_spk = result_spk.assign(skor=result_value)
result_spk = result_spk.sort_values(by=['skor'], ascending=False)
result_spk.drop_duplicates(subset=["nama"])

,nama,nama_jurusan,posisi,skor
6,SUGIHARTO,pengelolaan arsip dan rekaman informasi,kurator,2.344552
9,RAHMADIA ANISA PUTRI,teknologi rekayasa multimedia grafis,branding museum,2.311841
3,Kania Nurfitri Azzahra,arkeologi,kurator,2.205996
14,Naufal Aji Satria,administrasi publik,edukator,2.040517
2,Urmala Pingky Diavirma,ilmu administrasi negara,edukator,2.008491


In [63]:
result_spk

,nama,nama_jurusan,posisi,skor
6,SUGIHARTO,pengelolaan arsip dan rekaman informasi,kurator,2.344552
9,RAHMADIA ANISA PUTRI,teknologi rekayasa multimedia grafis,branding museum,2.311841
7,SUGIHARTO,pengelolaan arsip dan rekaman informasi,branding museum,2.244552
8,SUGIHARTO,pengelolaan arsip dan rekaman informasi,edukator,2.244552
10,RAHMADIA ANISA PUTRI,teknologi rekayasa multimedia grafis,edukator,2.211841
3,Kania Nurfitri Azzahra,arkeologi,kurator,2.205996
14,Naufal Aji Satria,administrasi publik,edukator,2.040517
2,Urmala Pingky Diavirma,ilmu administrasi negara,edukator,2.008491
4,Kania Nurfitri Azzahra,arkeologi,edukator,2.005996
11,RAHMADIA ANISA PUTRI,teknologi rekayasa multimedia grafis,kurator,1.911841
